In [7]:
!pip install flask flask-socketio pyngrok
!pip install flask flask-socketio eventlet
!pip install pyngrok

In [8]:
!ngrok authtoken 2gx07k2dnZkMR8dqD4nxLvyOmLJ_6ZUVQ6exmRovj113uQJXz
!ngrok restart

Authtoken saved to configuration file: /home/codespace/.config/ngrok/ngrok.yml
ngrok - tunnel local ports to public URLs and inspect traffic

USAGE:
  ngrok [command] [flags]

AUTHOR:
  ngrok - <support@ngrok.com>

COMMANDS: 
  config          update or migrate ngrok's configuration file
  http            start an HTTP tunnel
  tcp             start a TCP tunnel
  tunnel          start a tunnel for use with a tunnel-group backend

EXAMPLES: 
  ngrok http 80                                                 # secure public URL for port 80 web server
  ngrok http --domain baz.ngrok.dev 8080                        # port 8080 available at baz.ngrok.dev
  ngrok tcp 22                                                  # tunnel arbitrary TCP traffic to port 22
  ngrok http 80 --oauth=google --oauth-allow-email=foo@foo.com  # secure your app with oauth

Paid Features: 
  ngrok http 80 --domain mydomain.com                           # run ngrok with your own custom domain
  ngrok http 80 --allow-

In [2]:
import os
from flask import Flask, session, request
from flask_socketio import SocketIO, send, emit
from pyngrok import ngrok

# Initialize Flask app and SocketIO
app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

# Store chat messages
public_chat_history = []
private_messages = {}

# HTML template for the chat (unchanged, same as before)
chat_html = '''<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css">
  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
  <link href="https://fonts.googleapis.com/css2?family=Chakra+Petch:wght@300;400;500;600;700&display=swap" rel="stylesheet">
  <link href="https://unpkg.com/@bentoproject/core@0.2.3/dist/styles.css" rel="stylesheet">
  <title>Spider-Chat</title>
  <script src="https://cdn.socket.io/4.0.1/socket.io.min.js"></script>
  <style>
    body {
      font-family: 'Chakra Petch', sans-serif;
      margin: 0;
      display: flex;
      flex-direction: column;
      height: 100vh;
      background-color: #fef7dadc;
      color: #ffffff;
    }

    .chat-container {
      flex: 1;
      display: flex;
      flex-direction: column;
      padding: 10px;
    }

    #header {
      color: white;
      text-align: center;
      padding: 10px 0;
      font-size: 24px;
      position:sticky;
      top: 46px;
      z-index: 10000;

    }

    #hamburger {
      position: absolute;
      top: 50%;
      left: 10px;
      transform: translateY(-50%);
      cursor: pointer;
      z-index: 10000;
      width: 50px;
      height: 50px;
      display: flex;
      align-items: center;
      justify-content: center;
      border-radius: 50%;
      background: black;
      transition: all 0.3s ease;
    }

    .line {
      position: absolute;
      height: 2px;
      background-color: white;
      transition: all 0.3s ease;
    }

    .line1 {
      width: 20px;
      top: 40%;
    }

    .line2 {
      width: 10px;
      top: 60%;
    }

    #hamburger.open .line1 {
      transform: rotate(45deg);
      width: 20px;
      top: 50%;
    }

    #hamburger.open .line2 {
      transform: rotate(-45deg);
      width: 20px;
      top: 50%;
    }

    #messages {
      flex: 1;
      overflow-y: auto;
      margin: 10px 0;
    }

    .input-container {
      display: flex;
      padding: 10px 0;
      position:sticky;
      bottom:15px;
      background-color:#fef7da;
    }

    #myMessage {
      flex: 1;
      padding: 10px;
      border: 1px solid #333;
      border-radius: 40px;
      margin-right: 10px;
      background: #1e1e1e;
      color: #fff;
    }

    #sendbutton {
      background: linear-gradient(135deg, #d7d1b7, #000);
      color: white;
      border: none;
      padding: 10px;
      border-radius: 4px;
      cursor: pointer;
    }

    .navbar {
      background-color: #000000;
  border-left: 1px solid #333;
  position: fixed;
  bottom: -100%; /* initially hide it off-screen */
  width: 100%;
  height: 82%;
  transition: bottom ease-in-out 0.3s; /* transition based on bottom */
  z-index: 1000;
  border-radius: 50px;
  border-bottom-left-radius: 0;
border-bottom-right-radius: 0;
color: #fef7da;
      
    }

    .navbar.open {
      bottom: 0;
    }

    h2{
      color: #fef7da;
    }
    .navbar h2 {
      text-align: center;
      padding: 10px 0;
      margin: 10px;
      z-index: 1000;
      text-align: center;
padding: 20px;
margin-top: 20px;
border:3px solid #fef7da;
border-radius: 50px;
color: #fef7da;
font-size: 22px;
font-weight: 700;
    }

    #userList {
      list-style: none;
      padding: 0;
      margin: 0;
    }

    #userList li {
      padding: 10px;
      cursor: pointer;
      border-bottom: 1px solid #333;
      color: #fff;
    }

    .message {
      display: flex;
      align-items: center;
      margin-bottom: 10px;
    }

    .message .profile-pic {
      width: 40px;
      height: 40px;
      margin-right: 10px;
    }

    .message .profile-pic img {
      width: 100%;
      border-radius: 50%;
    }

    @media (max-width: 768px) {
      .navbar {
        width: 100%;
      }

      #header {
        font-size: 28px;
      }

      #myMessage {
        font-size: 18px;
      }

      #sendbutton {
        font-size: 18px;
      }

      #userList li {
        font-size: 18px;
      }

      .message {
        font-size: 18px;
      }
    }
    .texter{
text-align: center;
padding: 20px;
margin-top: -10px;
border:3px solid #000000;
border-radius: 50px;
color: #000;
font-size: 22px;
font-weight: 700;
position:sticky;
top:20px;
    background-color: #fef7da;
}
div{
color:#000;
}
  </style>
</head>
<body>
  <div class="chat-container">
    <div id="header">
      <div id="hamburger">
        <div class="line line1"></div>
        <div class="line line2"></div>
      </div>

    </div>
    <div class="texter">
      Spider-Chat
    </div>
    <div id="messages"></div>
    <div class="input-container">
      <input id="myMessage" placeholder="Type here" autocomplete="off">
      <button id="sendbutton"><i class="fas fa-paper-plane"></i></button>
    </div>
  </div>
  <div class="navbar" id="navbar">
    <h2>Users</h2>
    <ul id="userList"></ul>
  </div>
  <script type="text/javascript">
    document.addEventListener('DOMContentLoaded', (event) => {
      var socket = io.connect(window.location.href);
      var navbar = document.getElementById('navbar');
      var hamburger = document.getElementById('hamburger');

      function sendMessage(message, to) {
        if (message.trim() !== '') {
          socket.emit('message', {msg: message, to: to});
          document.getElementById('myMessage').value = '';
        }
      }

      function createMessageElement(data) {
        var messageElement = document.createElement('div');
        messageElement.classList.add('message', data.align);
        messageElement.innerHTML = `
          <div class="profile-pic">
            <img src="https://t4.ftcdn.net/jpg/00/64/67/27/360_F_64672736_U5kpdGs9keUll8CRQ3p3YaEv2M6qkVY5.jpg" alt="Profile">
          </div>
          <div>
            <strong>${data.username}:</strong> ${parseMessageContent(data.msg)}
          </div>`;
        if (data.to) {
          messageElement.innerHTML = `
            <div class="profile-pic">
              <img src="https://t4.ftcdn.net/jpg/00/64/67/27/360_F_64672736_U5kpdGs9keUll8CRQ3p3YaEv2M6qkVY5.jpg" alt="Profile">
            </div>
            <div>
              <strong>${data.username} to ${data.to}:</strong> ${parseMessageContent(data.msg)}
            </div>`;
        }
        return messageElement;
      }

      function parseMessageContent(message) {
        // Check for links
        var urlPattern = /(https?:\/\/[^\s]+)/g;
        message = message.replace(urlPattern, '<a href="$1" target="_blank">$1</a>');

        // Check for GIFs
        if (message.startsWith('/gif ')) {
          var gifQuery = message.replace('/gif ', '');
          return `<img src="https://api.giphy.com/v1/gifs/translate?api_key=YOUR_GIPHY_API_KEY&s=${encodeURIComponent(gifQuery)}" alt="GIF">`;
        }

        return message;
      }

      socket.on('message', function(data) {
        var messageElement = createMessageElement(data);
        document.getElementById('messages').appendChild(messageElement);
        document.getElementById('messages').scrollTop = document.getElementById('messages').scrollHeight;
      });

      socket.on('chat_history', function(data) {
        var messages = data.messages;
        messages.forEach(messageData => {
          var messageElement = createMessageElement(messageData);
          document.getElementById('messages').appendChild(messageElement);
          document.getElementById('messages').scrollTop = document.getElementById('messages').scrollHeight;
        });
      });

      socket.on('user_list', function(data) {
        var userList = document.getElementById('userList');
        userList.innerHTML = '';
        data.users.forEach(user => {
          var userElement = document.createElement('li');
          userElement.textContent = user.username;
          userElement.onclick = function() {
            var msg = prompt('Enter your message to ' + user.username);
            if (msg) {
              sendMessage(msg, user.id);
            }
          };
          userList.appendChild(userElement);
        });
      });

      document.getElementById('sendbutton').onclick = function() {
        var msg = document.getElementById('myMessage').value;
        sendMessage(msg, null);
      };

      document.getElementById('myMessage').addEventListener('keydown', function(event) {
        if (event.key === 'Enter') {
          var msg = document.getElementById('myMessage').value;
          sendMessage(msg, null);
        }
      });

      socket.on('connect', function() {
        let username = prompt('Enter your username:');
        socket.emit('set_username', {username: username});
        socket.emit('get_chat_history');
      });

      var navbar = document.getElementById('navbar');
      var hamburger = document.getElementById('hamburger');

      hamburger.onclick = function() {
        if (navbar.classList.contains('open')) {
          navbar.classList.remove('open');
          hamburger.classList.remove('open');
        } else {
          navbar.classList.add('open');
          hamburger.classList.add('open');
        }
      };
    });
  </script>
</body>
</html>



'''

users = {}
user_order = []

@app.route('/')
def chat():
    return chat_html

@socketio.on('connect')
def handle_connect():
    session['id'] = request.sid
    user_order.append(request.sid)
    print(f'{request.sid} connected')

@socketio.on('disconnect')
def handle_disconnect():
    user_id = session.get('id')
    if user_id in users:
        print(f'{users[user_id]["username"]} disconnected')
        del users[user_id]
        user_order.remove(user_id)
        emit_user_list()

@socketio.on('set_username')
def set_username(data):
    user_id = session.get('id')
    if user_id:
        username = data['username']
        users[user_id] = {
            'username': username
        }
        print(f'{username} has joined the chat')
        emit_user_list()

@socketio.on('message')
def handle_message(data):
    user_id = session.get('id')
    username = users.get(user_id, {}).get('username', 'Unknown')

    message_data = {
        'username': username,
        'msg': data['msg'],
        'align': 'right' if data['to'] else 'left',
        'to': None
    }

    if data['to']:
        recipient_id = data['to']
        recipient_username = users.get(recipient_id, {}).get('username', 'Unknown')
        message_data['to'] = recipient_username
        if recipient_id not in private_messages:
            private_messages[recipient_id] = []
        if user_id not in private_messages:
            private_messages[user_id] = []
        private_messages[recipient_id].append(message_data)
        private_messages[user_id].append(message_data)
        emit('message', message_data, room=recipient_id)
        emit('message', message_data, room=user_id)
    else:
        send(message_data, broadcast=True)
        public_chat_history.append(message_data)

@socketio.on('get_chat_history')
def handle_get_chat_history():
    user_id = session.get('id')
    if user_id:
        # Send public chat history
        emit('chat_history', {'messages': public_chat_history})
        # Send private messages for the user
        if user_id in private_messages:
            for private_message in private_messages[user_id]:
                emit('message', private_message, room=user_id)

def emit_user_list():
    user_list = [{'id': user_id, 'username': users[user_id]['username']} for user_id in users]
    emit('user_list', {'users': user_list}, broadcast=True)

# Start ngrok to expose the app
url = ngrok.connect(5000)
print(' * Tunnel URL:', url)

if __name__ == '__main__':
    socketio.run(app, port=5000, allow_unsafe_werkzeug=True)


 * Tunnel URL: NgrokTunnel: "https://7cf6-4-240-46-83.ngrok-free.app" -> "http://localhost:5000"
jcknXIso7Y3idrKVAAAB connected
spiderman has joined the chat
spiderman disconnected


In [2]:
import os
from flask import Flask, request, send_from_directory, render_template_string
from flask_socketio import SocketIO, emit
from werkzeug.utils import secure_filename
from pyngrok import ngrok

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

if not os.path.exists(app.config['UPLOAD_FOLDER']):
    os.makedirs(app.config['UPLOAD_FOLDER'])

public_chat_history = []
private_messages = {}
users = {}
user_order = []

# Function to delete all files in the uploads folder
def clear_uploaded_images():
    for filename in os.listdir(app.config['UPLOAD_FOLDER']):
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        os.remove(file_path)

# HTML template for the chat application
chat_html = '''<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css">
  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
  <link href="https://fonts.googleapis.com/css2?family=Chakra+Petch:wght@300;400;500;600;700&display=swap" rel="stylesheet">
  <link href="https://unpkg.com/@bentoproject/core@0.2.3/dist/styles.css" rel="stylesheet">
  <title>Spider-Chat</title>
  <script src="https://cdn.socket.io/4.0.1/socket.io.min.js"></script>
  <style>
    body {
      font-family: 'Chakra Petch', sans-serif;
      margin: 0;
      display: flex;
      flex-direction: column;
      height: 100vh;
      background-color: #fef7dadc;
      color: #ffffff;
    }
    .chat-container {
      flex: 1;
      display: flex;
      flex-direction: column;
      padding: 10px;
    }
    #header {
      color: white;
      text-align: center;
      padding: 10px 0;
      font-size: 24px;
      position:sticky;
      top: 46px;
      z-index: 10000;
    }
    #hamburger {
      position: absolute;
      top: 50%;
      left: 10px;
      transform: translateY(-50%);
      cursor: pointer;
      z-index: 10000;
      width: 50px;
      height: 50px;
      display: flex;
      align-items: center;
      justify-content: center;
      border-radius: 50%;
      background: black;
      transition: all 0.3s ease;
    }
    .line {
      position: absolute;
      height: 2px;
      background-color: white;
      transition: all 0.3s ease;
    }
    .line1 {
      width: 20px;
      top: 40%;
    }
    .line2 {
      width: 10px;
      top: 60%;
    }
    #hamburger.open .line1 {
      transform: rotate(45deg);
      width: 20px;
      top: 50%;
    }
    #hamburger.open .line2 {
      transform: rotate(-45deg);
      width: 20px;
      top: 50%;
    }
    #messages {
      flex: 1;
      overflow-y: auto;
      margin: 10px 0;
    }
    .input-container {
      display: flex;
      padding: 10px 0;
      position:sticky;
      bottom:15px;
      background-color:#fef7da;
    }
    #myMessage {
      flex: 1;
      padding: 10px;
      border: 1px solid #333;
      border-radius: 40px;
      margin-right: 10px;
      background: #1e1e1e;
      color: #fff;
    }
    #sendbutton {
      background: linear-gradient(135deg, #d7d1b7, #000);
      color: white;
      border: none;
      padding: 10px;
      border-radius: 4px;
      cursor: pointer;
    }
    #attachment {
      background: linear-gradient(135deg, #d7d1b7, #000);
      color: white;
      border: none;
      padding: 10px;
      border-radius: 4px;
      cursor: pointer;
      margin-right: 10px;
    }
    .navbar {
      background-color: #000000;
      border-left: 1px solid #333;
      position: fixed;
      bottom: -100%;
      width: 100%;
      height: 82%;
      transition: bottom ease-in-out 0.3s;
      z-index: 1000;
      border-radius: 50px;
      border-bottom-left-radius: 0;
      border-bottom-right-radius: 0;
      color: #fef7da;
    }
    .navbar.open {
      bottom: 0;
    }
    h2{
      color: #fef7da;
    }
    .navbar h2 {
      text-align: center;
      padding: 10px 0;
      margin: 10px;
      z-index: 1000;
      text-align: center;
      padding: 20px;
      margin-top: 20px;
      border:3px solid #fef7da;
      border-radius: 50px;
      color: #fef7da;
      font-size: 22px;
      font-weight: 700;
    }
    #userList {
      list-style: none;
      padding: 0;
      margin: 0;
    }
    #userList li {
      padding: 10px;
      cursor: pointer;
      border-bottom: 1px solid #333;
      color: #fff;
    }
    .message {
      display: flex;
      align-items: center;
      margin-bottom: 10px;
    }
    .message .profile-pic {
      width: 40px;
      height: 40px;
      margin-right: 10px;
    }
    .message .profile-pic img {
      width: 100%;
      border-radius: 50%;
    }
    @media (max-width: 768px) {
      .navbar {
        width: 100%;
      }
      #header {
        font-size: 28px;
      }
      #myMessage {
        font-size: 18px;
      }
      #sendbutton {
        font-size: 18px;
      }
      #userList li {
        font-size: 18px;
      }
      .message {
        font-size: 18px;
      }
    }
    .texter {
      text-align: center;
      padding: 20px;
      margin-top: -10px;
      border:3px solid #000000;
      border-radius: 50px;
      color: #000;
      font-size: 22px;
      font-weight: 700;
      position:sticky;
      top:20px;
      background-color: #fef7da;
    }
    div {
      color:#000;
    }
  </style>
</head>
<body>
  <div class="chat-container">
    <div id="header">
      <div id="hamburger">
        <div class="line line1"></div>
        <div class="line line2"></div>
      </div>
    </div>
    <div class="texter">
      Spider-Chat
    </div>
    <div id="messages"></div>
    <div class="input-container">
      <button id="attachment"><i class="fas fa-paperclip"></i></button>
      <div id="typingIndicator" style="display: none;">Username is typing...</div>
      <input id="myMessage" placeholder="Type here" autocomplete="off">
      <button id="sendbutton"><i class="fas fa-paper-plane"></i></button>
      <input type="file" id="fileInput" style="display: none;" accept="image/*">
    </div>
  </div>
  <div class="navbar" id="navbar">
    <h2>Users</h2>
    <ul id="userList"></ul>
  </div>
  <script type="text/javascript">
    document.addEventListener('DOMContentLoaded', (event) => {
      var socket = io.connect(window.location.href);
      var navbar = document.getElementById('navbar');
      var hamburger = document.getElementById('hamburger');
      var messageInput = document.getElementById('myMessage');

      function sendMessage(message, to) {
        if (message.trim() !== '') {
          const data = { message: message.trim(), to: to };
          socket.emit('send_message', data);
        }
      }

      function sendImage(file, to) {
        const formData = new FormData();
        formData.append('image', file);
        formData.append('to', to);
        fetch('/upload_image', {
          method: 'POST',
          body: formData
        }).then(response => response.json())
          .then(data => {
            if (data.url) {
              socket.emit('send_image', { url: data.url, to: to });
            }
          });
      }

      function renderMessage(message) {
        const messageElement = document.createElement('div');
        messageElement.classList.add('message');
        const profilePic = document.createElement('div');
        profilePic.classList.add('profile-pic');
        profilePic.innerHTML = '<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSPlwkUTsaWA_zSj0oFI7sbKYXqs-ih3b0_Bg&s' + message.username + '?d=identicon">';
        messageElement.appendChild(profilePic);
        const messageText = document.createElement('div');
        messageText.innerText = message.username + ': ' + message.text;
        messageElement.appendChild(messageText);
        document.getElementById('messages').appendChild(messageElement);
        document.getElementById('messages').scrollTop = document.getElementById('messages').scrollHeight;
      }

      function renderImage(message) {
        const messageElement = document.createElement('div');
        messageElement.classList.add('message');
        const profilePic = document.createElement('div');
        profilePic.classList.add('profile-pic');
        profilePic.innerHTML = '<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSPlwkUTsaWA_zSj0oFI7sbKYXqs-ih3b0_Bg&s' + message.username + '?d=identicon">';
        messageElement.appendChild(profilePic);
        const messageImage = document.createElement('div');
        messageImage.innerHTML = '<img src="' + message.url + '" style="max-width: 200px; border-radius: 10px;">';
        messageElement.appendChild(messageImage);
        document.getElementById('messages').appendChild(messageElement);
        document.getElementById('messages').scrollTop = document.getElementById('messages').scrollHeight;
      }

      socket.on('connect', () => {
        const username = prompt('Enter your username:');
        if (username) {
          socket.emit('set_username', { username: username });
        }
      });

      socket.on('user_list', (user_list) => {
        const userListElement = document.getElementById('userList');
        userListElement.innerHTML = '';
        user_list.forEach((user) => {
          const userElement = document.createElement('li');
          userElement.innerText = user;
          userElement.addEventListener('click', () => {
            const message = prompt('Enter your private message:');
            if (message) {
              sendMessage(message, user);
            }
          });
          userListElement.appendChild(userElement);
        });
      });

      socket.on('receive_message', (data) => {
        renderMessage(data);
      });

      socket.on('receive_image', (data) => {
        renderImage(data);
      });

      document.getElementById('sendbutton').addEventListener('click', () => {
        sendMessage(messageInput.value);
        messageInput.value = '';
      });

      document.getElementById('attachment').addEventListener('click', () => {
        document.getElementById('fileInput').click();
      });

      document.getElementById('fileInput').addEventListener('change', (event) => {
        const file = event.target.files[0];
        if (file) {
          sendImage(file);
        }
      });

      hamburger.addEventListener('click', () => {
        navbar.classList.toggle('open');
        hamburger.classList.toggle('open');
      });

      socket.on('chat_history', (history) => {
        history.forEach((message) => {
          if (message.type === 'text') {
            renderMessage(message);
          } else if (message.type === 'image') {
            renderImage(message);
          }
        });
      });

      // Send message on Enter key press
      messageInput.addEventListener('keydown', (event) => {
        if (event.key === 'Enter') {
          sendMessage(messageInput.value);
          messageInput.value = '';
        }
      });

      // Notify server on page unload
      window.addEventListener('beforeunload', () => {
        socket.emit('disconnect');
      });
    });
  </script>
</body>
</html>
'''

@app.route('/')
def index():
    return render_template_string(chat_html)

@socketio.on('set_username')
def handle_username(data):
    username = data['username']
    users[request.sid] = username
    user_order.append(username)
    private_messages[username] = []
    emit('user_list', user_order, broadcast=True)
    # Send the public chat history to the newly connected user
    emit('chat_history', public_chat_history, room=request.sid)

@socketio.on('send_message')
def handle_message(data):
    message = data['message']
    to = data.get('to')
    username = users.get(request.sid, 'Unknown')
    message_data = {'username': username, 'text': message, 'type': 'text'}

    if to:
        if to in users.values():
            recipient_sid = next(key for key, value in users.items() if value == to)
            private_messages[username].append(message_data)
            private_messages[to].append(message_data)
            emit('receive_message', message_data, room=recipient_sid)
        else:
            emit('receive_message', {'username': 'System', 'text': f'User {to} not found.', 'type': 'text'}, room=request.sid)
    else:
        public_chat_history.append(message_data)
        emit('receive_message', message_data, broadcast=True)

@socketio.on('send_image')
def handle_image(data):
    url = data['url']
    to = data.get('to')
    username = users.get(request.sid, 'Unknown')
    image_data = {'username': username, 'url': url, 'type': 'image'}

    if to:
        if to in users.values():
            recipient_sid = next(key for key, value in users.items() if value == to)
            private_messages[username].append(image_data)
            private_messages[to].append(image_data)
            emit('receive_image', image_data, room=recipient_sid)
        else:
            emit('receive_message', {'username': 'System', 'text': f'User {to} not found.', 'type': 'text'}, room=request.sid)
    else:
        public_chat_history.append(image_data)
        emit('receive_image', image_data, broadcast=True)

@socketio.on('disconnect')
def handle_disconnect():
    username = users.pop(request.sid, None)
    if username:
        user_order.remove(username)
        emit('user_list', user_order, broadcast=True)

@app.route('/upload_image', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return {'error': 'No file part'}
    file = request.files['image']
    if file.filename == '':
        return {'error': 'No selected file'}
    if file:
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        url = f'/uploads/{filename}'
        return {'url': url}
    return {'error': 'File upload failed'}

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

if __name__ == '__main__':
    clear_uploaded_images()
    port = 5000
    public_url = ngrok.connect(port)
    print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, port))
    socketio.run(app, port=port)


 * ngrok tunnel "NgrokTunnel: "https://c396-20-192-21-48.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000/"


In [1]:
import os
from flask import Flask, request, send_from_directory, render_template_string
from flask_socketio import SocketIO, emit
from werkzeug.utils import secure_filename
from pyngrok import ngrok

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

if not os.path.exists(app.config['UPLOAD_FOLDER']):
    os.makedirs(app.config['UPLOAD_FOLDER'])

public_chat_history = []
private_messages = {}
users = {}
user_order = []

# Function to delete all files in the uploads folder
def clear_uploaded_images():
    for filename in os.listdir(app.config['UPLOAD_FOLDER']):
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        os.remove(file_path)

# HTML template for the chat application
chat_html = '''<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css">
  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
  <link href="https://fonts.googleapis.com/css2?family=Chakra+Petch:wght@300;400;500;600;700&display=swap" rel="stylesheet">
  <link href="https://unpkg.com/@bentoproject/core@0.2.3/dist/styles.css" rel="stylesheet">
  <title>Spider-Chat</title>
  <script src="https://cdn.socket.io/4.0.1/socket.io.min.js"></script>
  <style>
    body {
      font-family: 'Chakra Petch', sans-serif;
      margin: 0;
      display: flex;
      flex-direction: column;
      height: 100vh;
      background-color: #fef7dadc;
      color: #ffffff;
    }
    .chat-container {
      flex: 1;
      display: flex;
      flex-direction: column;
      padding: 10px;
    }
    #header {
      color: white;
      text-align: center;
      padding: 10px 0;
      font-size: 24px;
      position:sticky;
      top: 46px;
      z-index: 10000;
    }
    #hamburger {
      position: absolute;
      top: 50%;
      left: 10px;
      transform: translateY(-50%);
      cursor: pointer;
      z-index: 10000;
      width: 50px;
      height: 50px;
      display: flex;
      align-items: center;
      justify-content: center;
      border-radius: 50%;
      background: black;
      transition: all 0.3s ease;
    }
    .line {
      position: absolute;
      height: 2px;
      background-color: white;
      transition: all 0.3s ease;
    }
    .line1 {
      width: 20px;
      top: 40%;
    }
    .line2 {
      width: 10px;
      top: 60%;
    }
    #hamburger.open .line1 {
      transform: rotate(45deg);
      width: 20px;
      top: 50%;
    }
    #hamburger.open .line2 {
      transform: rotate(-45deg);
      width: 20px;
      top: 50%;
    }
    #messages {
      flex: 1;
      overflow-y: auto;
      margin: 10px 0;
    }
    .input-container {
      display: flex;
      padding: 10px 0;
      position:sticky;
      bottom:15px;
      background-color:#fef7da;
    }
    #myMessage {
      flex: 1;
      padding: 10px;
      border: 1px solid #333;
      border-radius: 40px;
      margin-right: 10px;
      background: #1e1e1e;
      color: #fff;
    }
    #sendbutton {
      background: linear-gradient(135deg, #d7d1b7, #000);
      color: white;
      border: none;
      padding: 10px;
      border-radius: 4px;
      cursor: pointer;
    }
    #attachment {
      background: linear-gradient(135deg, #d7d1b7, #000);
      color: white;
      border: none;
      padding: 10px;
      border-radius: 4px;
      cursor: pointer;
      margin-right: 10px;
    }
    .navbar {
      background-color: #000000;
      border-left: 1px solid #333;
      position: fixed;
      bottom: -100%;
      width: 100%;
      height: 82%;
      transition: bottom ease-in-out 0.3s;
      z-index: 1000;
      border-radius: 50px;
      border-bottom-left-radius: 0;
      border-bottom-right-radius: 0;
      color: #fef7da;
    }
    .navbar.open {
      bottom: 0;
    }
    h2{
      color: #fef7da;
    }
    .navbar h2 {
      text-align: center;
      padding: 10px 0;
      margin: 10px;
      z-index: 1000;
      text-align: center;
      padding: 20px;
      margin-top: 20px;
      border:3px solid #fef7da;
      border-radius: 50px;
      color: #fef7da;
      font-size: 22px;
      font-weight: 700;
    }
    #userList {
      list-style: none;
      padding: 0;
      margin: 0;
    }
    #userList li {
      padding: 10px;
      cursor: pointer;
      border-bottom: 1px solid #333;
      color: #fff;
    }
    .message {
      display: flex;
      align-items: center;
      margin-bottom: 10px;
    }
    .message .profile-pic {
      width: 40px;
      height: 40px;
      margin-right: 10px;
    }
    .message .profile-pic img {
      width: 100%;
      border-radius: 50%;
    }
    #typing-indicator {
      color: #000;
      font-style: italic;
      padding: 0 10px;
      z-index: 900;
      margin-top:-20px
    }
    @media (max-width: 768px) {
      .navbar {
        width: 100%;
      }
      #header {
        font-size: 28px;
      }
      #myMessage {
        font-size: 18px;
      }
      #sendbutton {
        font-size: 18px;
      }
      #userList li {
        font-size: 18px;
      }
      .message {
        font-size: 18px;
      }
    }
    .texter {
      text-align: center;
      padding: 20px;
      margin-top: -10px;
      border:3px solid #000000;
      border-radius: 50px;
      color: #000;
      font-size: 22px;
      font-weight: 700;
      position:sticky;
      top:20px;
      background-color: #fef7da;
    }
    div {
      color:#000;
    }
    #scrollBottom {
  background: linear-gradient(135deg, #000, #d7d1b7); /* Inverted colors */
  color: white;
  border: none;
  padding: 10px;
  border-radius: 50%; /* Make it circular */
  cursor: pointer;
  width: 40px;
  height: 40px;
  display: flex;
  align-items: center;
  justify-content: center;
  margin-left: 10px; /* Add some space between buttons */
}

  </style>
</head>
<body>
  <div class="chat-container">
    <div id="header">
      <div id="hamburger">
        <div class="line line1"></div>
        <div class="line line2"></div>
      </div>
    </div>
    <div class="texter">
      Spider-Chat
    </div>
    <div id="messages"></div>
    <div id="typing-indicator"></div>
<div class="input-container">
  <button id="attachment"><i class="fas fa-paperclip"></i></button>
  <input id="myMessage" placeholder="Type here" autocomplete="off">
  <button id="sendbutton"><i class="fas fa-paper-plane"></i></button>
  <button id="scrollBottom"><i class="fas fa-arrow-up"></i></button> <!-- Scroll to bottom button -->
  <input type="file" id="fileInput" style="display: none;" accept="image/*">
</div>

  </div>
  <div class="navbar" id="navbar">
    <h2>Users</h2>
    <ul id="userList"></ul>
  </div>
  <script type="text/javascript">
    document.addEventListener('DOMContentLoaded', (event) => {
      var socket = io.connect(window.location.href);
      var navbar = document.getElementById('navbar');
      var hamburger = document.getElementById('hamburger');
      var messageInput = document.getElementById('myMessage');
      var typingIndicator = document.getElementById('typing-indicator');
      var typingTimeout;

      function sendMessage(message, to) {
        if (message.trim() !== '') {
          const data = { message: message.trim(), to: to };
          socket.emit('send_message', data);
        }
      }
      
      document.getElementById('scrollBottom').addEventListener('click', () => {
          const messagesContainer = document.getElementById('messages');
          messagesContainer.scrollTop = messagesContainer.scrollHeight;
      });


      function sendImage(file, to) {
        const formData = new FormData();
        formData.append('image', file);
        formData.append('to', to);
        fetch('/upload_image', {
          method: 'POST',
          body: formData
        }).then(response => response.json())
        .then(data => {
          if (data.url) {
            socket.emit('send_image', { url: data.url, to: to });
          }
        });
      }

      function renderMessage(message) {
        const messageElement = document.createElement('div');
        messageElement.classList.add('message');
        const profilePic = document.createElement('div');
        profilePic.classList.add('profile-pic');
        profilePic.innerHTML = '<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSPlwkUTsaWA_zSj0oFI7sbKYXqs-ih3b0_Bg&s' + message.username + '?d=identicon">';
        messageElement.appendChild(profilePic);
        const messageText = document.createElement('div');
        messageText.innerText = message.username + ': ' + message.text;
        messageElement.appendChild(messageText);
        document.getElementById('messages').appendChild(messageElement);
        document.getElementById('messages').scrollTop = document.getElementById('messages').scrollHeight;
      }

      function renderImage(message) {
        const messageElement = document.createElement('div');
        messageElement.classList.add('message');
        const profilePic = document.createElement('div');
        profilePic.classList.add('profile-pic');
        profilePic.innerHTML = '<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSPlwkUTsaWA_zSj0oFI7sbKYXqs-ih3b0_Bg&s' + message.username + '?d=identicon">';
        messageElement.appendChild(profilePic);
        const messageImage = document.createElement('div');
        messageImage.innerHTML = '<img src="' + message.url + '" style="max-width: 200px; border-radius: 10px;">';
        messageElement.appendChild(messageImage);
        document.getElementById('messages').appendChild(messageElement);
        document.getElementById('messages').scrollTop = document.getElementById('messages').scrollHeight;
      }

      socket.on('connect', () => {
        const username = prompt('Enter your username:');
        if (username) {
          socket.emit('set_username', { username: username });
        }
      });

      socket.on('user_list', (user_list) => {
        const userListElement = document.getElementById('userList');
        userListElement.innerHTML = '';
        user_list.forEach((user) => {
          const userElement = document.createElement('li');
          userElement.innerText = user;
          userElement.addEventListener('click', () => {
            const message = prompt('Enter your private message:');
            if (message) {
              sendMessage(message, user);
            }
          });
          userListElement.appendChild(userElement);
        });
      });

      socket.on('receive_message', (data) => {
        renderMessage(data);
      });

      socket.on('receive_image', (data) => {
        renderImage(data);
      });

      document.getElementById('sendbutton').addEventListener('click', () => {
        sendMessage(messageInput.value);
        messageInput.value = '';
        socket.emit('stop_typing');
      });

      document.getElementById('attachment').addEventListener('click', () => {
        document.getElementById('fileInput').click();
      });

      document.getElementById('fileInput').addEventListener('change', (event) => {
        const file = event.target.files[0];
        if (file) {
          sendImage(file);
        }
      });

      hamburger.addEventListener('click', () => {
        navbar.classList.toggle('open');
        hamburger.classList.toggle('open');
      });

      socket.on('chat_history', (history) => {
        history.forEach((message) => {
          if (message.type === 'text') {
            renderMessage(message);
          } else if (message.type === 'image') {
            renderImage(message);
          }
        });
      });

      // Send message on Enter key press
      messageInput.addEventListener('keydown', (event) => {
        if (event.key === 'Enter') {
          sendMessage(messageInput.value);
          messageInput.value = '';
          socket.emit('stop_typing');
        } else {
          socket.emit('typing');
          clearTimeout(typingTimeout);
          typingTimeout = setTimeout(() => {
            socket.emit('stop_typing');
          }, 1000);
        }
      });

      // Notify server on page unload
      window.addEventListener('beforeunload', () => {
        socket.emit('disconnect');
      });

      socket.on('typing', (data) => {
        typingIndicator.innerText = data.username + ' is typing...';
        typingIndicator.style.display = 'block';
      });

      socket.on('stop_typing', () => {
        typingIndicator.style.display = 'none';
      });
    });
  </script>
</body>
</html>'''

@app.route('/')
def index():
    return render_template_string(chat_html)

@app.route('/upload_image', methods=['POST'])
def upload_image():
    file = request.files['image']
    if file:
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        return {'url': f'/uploads/{filename}'}
    return {'url': None}

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

@socketio.on('send_message')
def handle_send_message(data):
    username = users.get(request.sid, 'Unknown')
    message = {'type': 'text', 'username': username, 'text': data['message']}
    public_chat_history.append(message)
    emit('receive_message', message, broadcast=True)

@socketio.on('send_image')
def handle_send_image(data):
    username = users.get(request.sid, 'Unknown')
    message = {'type': 'image', 'username': username, 'url': data['url']}
    public_chat_history.append(message)
    emit('receive_image', message, broadcast=True)

@socketio.on('set_username')
def handle_set_username(data):
    users[request.sid] = data['username']
    user_order.append(data['username'])
    emit('user_list', user_order, broadcast=True)
    emit('chat_history', public_chat_history)

@socketio.on('typing')
def handle_typing():
    username = users.get(request.sid, 'Unknown')
    emit('typing', {'username': username}, broadcast=True, include_self=False)

@socketio.on('stop_typing')
def handle_stop_typing():
    emit('stop_typing', broadcast=True, include_self=False)

@socketio.on('disconnect')
def handle_disconnect():
    if request.sid in users:
        del users[request.sid]
    emit('user_list', list(users.values()), broadcast=True)

if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(" * Tunnel URL:", public_url)
    socketio.run(app, port=5000)


 * Tunnel URL: NgrokTunnel: "https://666c-20-192-21-48.ngrok-free.app" -> "http://localhost:5000"


t=2024-08-10T13:13:05+0000 lvl=warn msg="failed to check for update" obj=updater err="Post \"https://update.equinox.io/check\": context deadline exceeded"


In [3]:
pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install Flask SQLAlchemy Flask-WTF Flask-Login Flask-Migrate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 6.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 3.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.0 MB/s eta 0:00:00eta 0:00:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from flask import Flask, render_template_string, request, session, redirect, url_for
from flask_socketio import SocketIO, emit
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///app.db'
db = SQLAlchemy(app)
socketio = SocketIO(app)

# Models
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(150), unique=True, nullable=False)
    password = db.Column(db.String(150), nullable=False)

class Post(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    message = db.Column(db.String(500), nullable=False)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    user = db.relationship('User', backref=db.backref('posts', lazy=True))

# Inline templates and static files
index_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Social App</title>
    <style>
        body {
            font-family: Arial, sans-serif;
        }
    </style>
</head>
<body>
    <h1>Welcome to My Social App</h1>
    <div id="feed"></div>
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>
    <script>
        const socket = io.connect('http://' + document.domain + ':' + location.port);
        socket.on('connect', () => {
            console.log('Connected to server');
        });
        socket.on('new_post', (data) => {
            const feed = document.getElementById('feed');
            const post = document.createElement('div');
            post.textContent = data.message;
            feed.appendChild(post);
        });
        function createPost(message) {
            socket.emit('create_post', { message: message });
        }
    </script>
</body>
</html>
"""

@app.route('/')
def index():
    return render_template_string(index_html)

@app.route('/login', methods=['GET', 'POST'])
def login():
    # Implement login logic
    pass

@app.route('/signup', methods=['GET', 'POST'])
def signup():
    # Implement signup logic
    pass

@socketio.on('create_post')
def handle_create_post(json):
    message = json['message']
    post = Post(message=message)
    db.session.add(post)
    db.session.commit()
    emit('new_post', {'message': message}, broadcast=True)

if __name__ == '__main__':
    with app.app_context():
        db.create_all()
    socketio.run(app)


In [2]:
import os
from flask import Flask, request, send_from_directory, render_template_string
from flask_socketio import SocketIO, emit
from werkzeug.utils import secure_filename
from pyngrok import ngrok

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

if not os.path.exists(app.config['UPLOAD_FOLDER']):
    os.makedirs(app.config['UPLOAD_FOLDER'])

public_chat_history = []
private_messages = {}
users = {}
user_order = []

# Function to delete all files in the uploads folder
def clear_uploaded_images():
    for filename in os.listdir(app.config['UPLOAD_FOLDER']):
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        os.remove(file_path)

# HTML template for the chat application
chat_html = '''<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css">
  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
  <link href="https://fonts.googleapis.com/css2?family=Chakra+Petch:wght@300;400;500;600;700&display=swap" rel="stylesheet">
  <link href="https://unpkg.com/@bentoproject/core@0.2.3/dist/styles.css" rel="stylesheet">
  <title>Spider-Chat</title>
  <script src="https://cdn.socket.io/4.0.1/socket.io.min.js"></script>
  <style>
    body {
      font-family: 'Chakra Petch', sans-serif;
      margin: 0;
      display: flex;
      flex-direction: column;
      height: 100vh;
      background-color: #fef7dadc;
      color: #ffffff;
    }
    .chat-container {
      flex: 1;
      display: flex;
      flex-direction: column;
      padding: 10px;
    }
    #header {
      color: white;
      text-align: center;
      padding: 10px 0;
      font-size: 24px;
      position:sticky;
      top: 46px;
      z-index: 10000;
    }
    #hamburger {
      position: absolute;
      top: 50%;
      left: 10px;
      transform: translateY(-50%);
      cursor: pointer;
      z-index: 10000;
      width: 50px;
      height: 50px;
      display: flex;
      align-items: center;
      justify-content: center;
      border-radius: 50%;
      background: black;
      transition: all 0.3s ease;
    }
    .line {
      position: absolute;
      height: 2px;
      background-color: white;
      transition: all 0.3s ease;
    }
    .line1 {
      width: 20px;
      top: 40%;
    }
    .line2 {
      width: 10px;
      top: 60%;
    }
    #hamburger.open .line1 {
      transform: rotate(45deg);
      width: 20px;
      top: 50%;
    }
    #hamburger.open .line2 {
      transform: rotate(-45deg);
      width: 20px;
      top: 50%;
    }
    #messages {
      flex: 1;
      overflow-y: auto;
      margin: 10px 0;
    }
    .input-container {
      display: flex;
      padding: 10px 0;
      position:sticky;
      bottom:15px;
      background-color:#fef7da;
    }
    #myMessage {
      flex: 1;
      padding: 10px;
      border: 1px solid #333;
      border-radius: 40px;
      margin-right: 10px;
      background: #1e1e1e;
      color: #fff;
    }
    #sendbutton {
      background: linear-gradient(135deg, #d7d1b7, #000);
      color: white;
      border: none;
      padding: 10px;
      border-radius: 4px;
      cursor: pointer;
    }
    #attachment {
      background: linear-gradient(135deg, #d7d1b7, #000);
      color: white;
      border: none;
      padding: 10px;
      border-radius: 4px;
      cursor: pointer;
      margin-right: 10px;
    }
    .navbar {
      background-color: #000000;
      border-left: 1px solid #333;
      position: fixed;
      bottom: -100%;
      width: 100%;
      height: 82%;
      transition: bottom ease-in-out 0.3s;
      z-index: 1000;
      border-radius: 50px;
      border-bottom-left-radius: 0;
      border-bottom-right-radius: 0;
      color: #fef7da;
    }
    .navbar.open {
      bottom: 0;
    }
    h2{
      color: #fef7da;
    }
    .navbar h2 {
      text-align: center;
      padding: 10px 0;
      margin: 10px;
      z-index: 1000;
      text-align: center;
      padding: 20px;
      margin-top: 20px;
      border:3px solid #fef7da;
      border-radius: 50px;
      color: #fef7da;
      font-size: 22px;
      font-weight: 700;
    }
    #userList {
      list-style: none;
      padding: 0;
      margin: 0;
    }
    #userList li {
      padding: 10px;
      cursor: pointer;
      border-bottom: 1px solid #333;
      color: #fff;
    }
    .message {
      display: flex;
      align-items: center;
      margin-bottom: 10px;
    }
    .message .profile-pic {
      width: 40px;
      height: 40px;
      margin-right: 10px;
    }
    .message .profile-pic img {
      width: 100%;
      border-radius: 50%;
    }
    #typing-indicator {
      color: #000;
      font-style: italic;
      padding: 0 10px;
      z-index: 1000;
      margin-top: -20px;
    }
    @media (max-width: 768px) {
      .navbar {
        width: 100%;
      }
      #header {
        font-size: 28px;
      }
      #myMessage {
        font-size: 18px;
      }
      #sendbutton {
        font-size: 18px;
      }
      #userList li {
        font-size: 18px;
      }
      .message {
        font-size: 18px;
      }
    }
    .texter {
      text-align: center;
      padding: 20px;
      margin-top: -10px;
      border:3px solid #000000;
      border-radius: 50px;
      color: #000;
      font-size: 22px;
      font-weight: 700;
      position:sticky;
      top:20px;
      background-color: #fef7da;
    }
    div {
      color:#000;
    }
  </style>
</head>
<body>
  <div class="chat-container">
    <div id="header">
      <div id="hamburger">
        <div class="line line1"></div>
        <div class="line line2"></div>
      </div>
    </div>
    <div class="texter">
      Spider-Chat
    </div>
    <div id="messages"></div>
    <div id="typing-indicator"></div>
    <div class="input-container">
      <button id="attachment"><i class="fas fa-paperclip"></i></button>
      <input id="myMessage" placeholder="Type here" autocomplete="off">
      <button id="sendbutton"><i class="fas fa-paper-plane"></i></button>
      <input type="file" id="fileInput" style="display: none;" accept="image/*">
    </div>
  </div>
  <div class="navbar" id="navbar">
    <h2>Users</h2>
    <ul id="userList"></ul>
  </div>
  <script type="text/javascript">
    document.addEventListener('DOMContentLoaded', (event) => {
      var socket = io.connect(window.location.href);
      var navbar = document.getElementById('navbar');
      var hamburger = document.getElementById('hamburger');
      var messageInput = document.getElementById('myMessage');
      var typingIndicator = document.getElementById('typing-indicator');
      var typingTimeout;

      function sendMessage(message, to) {
        if (message.trim() !== '') {
          const data = { message: message.trim(), to: to };
          socket.emit('send_message', data);
        }
      }

      function sendImage(file, to) {
        const formData = new FormData();
        formData.append('image', file);
        formData.append('to', to);
        fetch('/upload_image', {
          method: 'POST',
          body: formData
        }).then(response => response.json())
        .then(data => {
          if (data.url) {
            socket.emit('send_image', { url: data.url, to: to });
          }
        });
      }

      function renderMessage(message) {
        const messageElement = document.createElement('div');
        messageElement.classList.add('message');
        const profilePic = document.createElement('div');
        profilePic.classList.add('profile-pic');
        profilePic.innerHTML = '<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSPlwkUTsaWA_zSj0oFI7sbKYXqs-ih3b0_Bg&s' + message.username + '?d=identicon">';
        messageElement.appendChild(profilePic);
        const messageText = document.createElement('div');
        messageText.innerText = message.username + ': ' + message.text;
        messageElement.appendChild(messageText);
        document.getElementById('messages').appendChild(messageElement);
        document.getElementById('messages').scrollTop = document.getElementById('messages').scrollHeight;
      }

      function renderImage(message) {
        const messageElement = document.createElement('div');
        messageElement.classList.add('message');
        const profilePic = document.createElement('div');
        profilePic.classList.add('profile-pic');
        profilePic.innerHTML = '<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSPlwkUTsaWA_zSj0oFI7sbKYXqs-ih3b0_Bg&s' + message.username + '?d=identicon">';
        messageElement.appendChild(profilePic);
        const messageImage = document.createElement('div');
        messageImage.innerHTML = '<img src="' + message.url + '" style="max-width: 200px; border-radius: 10px;">';
        messageElement.appendChild(messageImage);
        document.getElementById('messages').appendChild(messageElement);
        document.getElementById('messages').scrollTop = document.getElementById('messages').scrollHeight;
      }

      socket.on('connect', () => {
        const username = prompt('Enter your username:');
        if (username) {
          socket.emit('set_username', { username: username });
        }
      });

      socket.on('user_list', (user_list) => {
        const userListElement = document.getElementById('userList');
        userListElement.innerHTML = '';
        user_list.forEach((user) => {
          const userElement = document.createElement('li');
          userElement.innerText = user;
          userElement.addEventListener('click', () => {
            const message = prompt('Enter your private message:');
            if (message) {
              sendMessage(message, user);
            }
          });
          userListElement.appendChild(userElement);
        });
      });

      socket.on('receive_message', (data) => {
        renderMessage(data);
      });

      socket.on('receive_image', (data) => {
        renderImage(data);
      });

      document.getElementById('sendbutton').addEventListener('click', () => {
        sendMessage(messageInput.value);
        messageInput.value = '';
        socket.emit('stop_typing');
      });

      document.getElementById('attachment').addEventListener('click', () => {
        document.getElementById('fileInput').click();
      });

      document.getElementById('fileInput').addEventListener('change', (event) => {
        const file = event.target.files[0];
        if (file) {
          sendImage(file);
        }
      });

      hamburger.addEventListener('click', () => {
        navbar.classList.toggle('open');
        hamburger.classList.toggle('open');
      });

      socket.on('chat_history', (history) => {
        history.forEach((message) => {
          if (message.type === 'text') {
            renderMessage(message);
          } else if (message.type === 'image') {
            renderImage(message);
          }
        });
      });

      // Send message on Enter key press
      messageInput.addEventListener('keydown', (event) => {
        if (event.key === 'Enter') {
          sendMessage(messageInput.value);
          messageInput.value = '';
          socket.emit('stop_typing');
        } else {
          socket.emit('typing');
          clearTimeout(typingTimeout);
          typingTimeout = setTimeout(() => {
            socket.emit('stop_typing');
          }, 1000);
        }
      });

      // Notify server on page unload
      window.addEventListener('beforeunload', () => {
        socket.emit('disconnect');
      });

      socket.on('typing', (data) => {
        typingIndicator.innerText = data.username + ' is typing...';
        typingIndicator.style.display = 'block';
      });

      socket.on('stop_typing', () => {
        typingIndicator.style.display = 'none';
      });
    });
  </script>
</body>
</html>'''

@app.route('/')
def index():
    return render_template_string(chat_html)

@app.route('/upload_image', methods=['POST'])
def upload_image():
    file = request.files['image']
    if file:
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        return {'url': f'/uploads/{filename}'}
    return {'url': None}

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

@socketio.on('send_message')
def handle_send_message(data):
    username = users.get(request.sid, 'Unknown')
    message = {'type': 'text', 'username': username, 'text': data['message']}
    public_chat_history.append(message)
    emit('receive_message', message, broadcast=True)

@socketio.on('send_image')
def handle_send_image(data):
    username = users.get(request.sid, 'Unknown')
    message = {'type': 'image', 'username': username, 'url': data['url']}
    public_chat_history.append(message)
    emit('receive_image', message, broadcast=True)

@socketio.on('set_username')
def handle_set_username(data):
    users[request.sid] = data['username']
    user_order.append(data['username'])
    emit('user_list', user_order, broadcast=True)
    emit('chat_history', public_chat_history)

@socketio.on('typing')
def handle_typing():
    username = users.get(request.sid, 'Unknown')
    emit('typing', {'username': username}, broadcast=True, include_self=False)

@socketio.on('stop_typing')
def handle_stop_typing():
    emit('stop_typing', broadcast=True, include_self=False)

@socketio.on('disconnect')
def handle_disconnect():
    if request.sid in users:
        del users[request.sid]
    emit('user_list', list(users.values()), broadcast=True)

if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(" * Tunnel URL:", public_url)
    socketio.run(app, port=5000)


 * Tunnel URL: NgrokTunnel: "https://9265-20-192-21-48.ngrok-free.app" -> "http://localhost:5000"


In [3]:
import os
from flask import Flask, request, send_from_directory, render_template_string
from flask_socketio import SocketIO, emit
from werkzeug.utils import secure_filename
from pyngrok import ngrok

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

if not os.path.exists(app.config['UPLOAD_FOLDER']):
    os.makedirs(app.config['UPLOAD_FOLDER'])

public_chat_history = []
private_messages = {}
users = {}
user_order = []

# Function to delete all files in the uploads folder
def clear_uploaded_images():
    for filename in os.listdir(app.config['UPLOAD_FOLDER']):
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        os.remove(file_path)

# HTML template for the chat application
chat_html = '''<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css">
  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
  <link href="https://fonts.googleapis.com/css2?family=Chakra+Petch:wght@300;400;500;600;700&display=swap" rel="stylesheet">
  <link href="https://unpkg.com/@bentoproject/core@0.2.3/dist/styles.css" rel="stylesheet">
  <title>Spider-Chat</title>
  <script src="https://cdn.socket.io/4.0.1/socket.io.min.js"></script>
  <style>
    body {
      font-family: 'Chakra Petch', sans-serif;
      margin: 0;
      display: flex;
      flex-direction: column;
      height: 100vh;
      background-color: #fef7dadc;
      color: #ffffff;
    }
    .chat-container {
      flex: 1;
      display: flex;
      flex-direction: column;
      padding: 10px;
    }
    #header {
      color: white;
      text-align: center;
      padding: 10px 0;
      font-size: 24px;
      position:sticky;
      top: 46px;
      z-index: 10000;
    }
    #hamburger {
      position: absolute;
      top: 50%;
      left: 10px;
      transform: translateY(-50%);
      cursor: pointer;
      z-index: 10000;
      width: 50px;
      height: 50px;
      display: flex;
      align-items: center;
      justify-content: center;
      border-radius: 50%;
      background: black;
      transition: all 0.3s ease;
    }
    .line {
      position: absolute;
      height: 2px;
      background-color: white;
      transition: all 0.3s ease;
    }
    .line1 {
      width: 20px;
      top: 40%;
    }
    .line2 {
      width: 10px;
      top: 60%;
    }
    #hamburger.open .line1 {
      transform: rotate(45deg);
      width: 20px;
      top: 50%;
    }
    #hamburger.open .line2 {
      transform: rotate(-45deg);
      width: 20px;
      top: 50%;
    }
    #messages {
      flex: 1;
      overflow-y: auto;
      margin: 10px 0;
    }
    .input-container {
      display: flex;
      padding: 10px 0;
      position:sticky;
      bottom:15px;
      background-color:#fef7da;
    }
    #myMessage {
      flex: 1;
      padding: 10px;
      border: 1px solid #333;
      border-radius: 40px;
      margin-right: 10px;
      background: #1e1e1e;
      color: #fff;
    }
    #sendbutton {
      background: linear-gradient(135deg, #d7d1b7, #000);
      color: white;
      border: none;
      padding: 10px;
      border-radius: 4px;
      cursor: pointer;
    }
    #attachment {
      background: linear-gradient(135deg, #d7d1b7, #000);
      color: white;
      border: none;
      padding: 10px;
      border-radius: 4px;
      cursor: pointer;
      margin-right: 10px;
    }
    .navbar {
      background-color: #000000;
      border-left: 1px solid #333;
      position: fixed;
      bottom: -100%;
      width: 100%;
      height: 82%;
      transition: bottom ease-in-out 0.3s;
      z-index: 1000;
      border-radius: 50px;
      border-bottom-left-radius: 0;
      border-bottom-right-radius: 0;
      color: #fef7da;
    }
    .navbar.open {
      bottom: 0;
    }
    h2{
      color: #fef7da;
    }
    .navbar h2 {
      text-align: center;
      padding: 10px 0;
      margin: 10px;
      z-index: 1000;
      text-align: center;
      padding: 20px;
      margin-top: 20px;
      border:3px solid #fef7da;
      border-radius: 50px;
      color: #fef7da;
      font-size: 22px;
      font-weight: 700;
    }
    #userList {
      list-style: none;
      padding: 0;
      margin: 0;
    }
    #userList li {
      padding: 10px;
      cursor: pointer;
      border-bottom: 1px solid #333;
      color: #fff;
    }
    .message {
      display: flex;
      align-items: center;
      margin-bottom: 10px;
    }
    .message .profile-pic {
      width: 40px;
      height: 40px;
      margin-right: 10px;
    }
    .message .profile-pic img {
      width: 100%;
      border-radius: 50%;
    }
    @media (max-width: 768px) {
      .navbar {
        width: 100%;
      }
      #header {
        font-size: 28px;
      }
      #myMessage {
        font-size: 18px;
      }
      #sendbutton {
        font-size: 18px;
      }
      #userList li {
        font-size: 18px;
      }
      .message {
        font-size: 18px;
      }
    }
    .texter {
      text-align: center;
      padding: 20px;
      margin-top: -10px;
      border:3px solid #000000;
      border-radius: 50px;
      color: #000;
      font-size: 22px;
      font-weight: 700;
      position:sticky;
      top:20px;
      background-color: #fef7da;
    }
    div {
      color:#000;
    }
  </style>
</head>
<body>
  <div class="chat-container">
    <div id="header">
      <div id="hamburger">
        <div class="line line1"></div>
        <div class="line line2"></div>
      </div>
    </div>
    <div class="texter">
      Spider-Chat
    </div>
    <div id="messages"></div>
    <div class="input-container">
      <button id="attachment"><i class="fas fa-paperclip"></i></button>
      <input id="myMessage" placeholder="Type here" autocomplete="off">
      <button id="sendbutton"><i class="fas fa-paper-plane"></i></button>
      <input type="file" id="fileInput" style="display: none;" accept="image/*">
    </div>
  </div>
  <div class="navbar" id="navbar">
    <h2>Users</h2>
    <ul id="userList"></ul>
  </div>
  <script type="text/javascript">
    document.addEventListener('DOMContentLoaded', (event) => {
      var socket = io.connect(window.location.href);
      var navbar = document.getElementById('navbar');
      var hamburger = document.getElementById('hamburger');
      var messageInput = document.getElementById('myMessage');

      function sendMessage(message, to) {
        if (message.trim() !== '') {
          const data = { message: message.trim(), to: to };
          socket.emit('send_message', data);
        }
      }

      function sendImage(file, to) {
        const formData = new FormData();
        formData.append('image', file);
        formData.append('to', to);
        fetch('/upload_image', {
          method: 'POST',
          body: formData
        }).then(response => response.json())
          .then(data => {
            if (data.url) {
              socket.emit('send_image', { url: data.url, to: to });
            }
          });
      }

      function renderMessage(message) {
        const messageElement = document.createElement('div');
        messageElement.classList.add('message');
        const profilePic = document.createElement('div');
        profilePic.classList.add('profile-pic');
        profilePic.innerHTML = '<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSPlwkUTsaWA_zSj0oFI7sbKYXqs-ih3b0_Bg&s' + message.username + '?d=identicon">';
        messageElement.appendChild(profilePic);
        const messageText = document.createElement('div');
        messageText.innerText = message.username + ': ' + message.text;
        messageElement.appendChild(messageText);
        document.getElementById('messages').appendChild(messageElement);
        document.getElementById('messages').scrollTop = document.getElementById('messages').scrollHeight;
      }

      function renderImage(message) {
        const messageElement = document.createElement('div');
        messageElement.classList.add('message');
        const profilePic = document.createElement('div');
        profilePic.classList.add('profile-pic');
        profilePic.innerHTML = '<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSPlwkUTsaWA_zSj0oFI7sbKYXqs-ih3b0_Bg&s' + message.username + '?d=identicon">';
        messageElement.appendChild(profilePic);
        const messageImage = document.createElement('div');
        messageImage.innerHTML = '<img src="' + message.url + '" style="max-width: 200px; border-radius: 10px;">';
        messageElement.appendChild(messageImage);
        document.getElementById('messages').appendChild(messageElement);
        document.getElementById('messages').scrollTop = document.getElementById('messages').scrollHeight;
      }

      socket.on('connect', () => {
        const username = prompt('Enter your username:');
        if (username) {
          socket.emit('set_username', { username: username });
        }
      });

      socket.on('user_list', (user_list) => {
        const userListElement = document.getElementById('userList');
        userListElement.innerHTML = '';
        user_list.forEach((user) => {
          const userElement = document.createElement('li');
          userElement.innerText = user;
          userElement.addEventListener('click', () => {
            const message = prompt('Enter your private message:');
            if (message) {
              sendMessage(message, user);
            }
          });
          userListElement.appendChild(userElement);
        });
      });

      socket.on('receive_message', (data) => {
        renderMessage(data);
      });

      socket.on('receive_image', (data) => {
        renderImage(data);
      });

      document.getElementById('sendbutton').addEventListener('click', () => {
        sendMessage(messageInput.value);
        messageInput.value = '';
      });

      document.getElementById('attachment').addEventListener('click', () => {
        document.getElementById('fileInput').click();
      });

      document.getElementById('fileInput').addEventListener('change', (event) => {
        const file = event.target.files[0];
        if (file) {
          sendImage(file);
        }
      });

      hamburger.addEventListener('click', () => {
        navbar.classList.toggle('open');
        hamburger.classList.toggle('open');
      });

      socket.on('chat_history', (history) => {
        history.forEach((message) => {
          if (message.type === 'text') {
            renderMessage(message);
          } else if (message.type === 'image') {
            renderImage(message);
          }
        });
      });

      // Send message on Enter key press
      messageInput.addEventListener('keydown', (event) => {
        if (event.key === 'Enter') {
          sendMessage(messageInput.value);
          messageInput.value = '';
        }
      });

      // Notify server on page unload
      window.addEventListener('beforeunload', () => {
        socket.emit('disconnect');
      });
    });
  </script>
</body>
</html>
'''

@app.route('/')
def index():
    return render_template_string(chat_html)

@socketio.on('set_username')
def handle_username(data):
    username = data['username']
    users[request.sid] = username
    user_order.append(username)
    private_messages[username] = []
    emit('user_list', user_order, broadcast=True)
    # Send the public chat history to the newly connected user
    emit('chat_history', public_chat_history, room=request.sid)

@socketio.on('send_message')
def handle_message(data):
    message = data['message']
    to = data.get('to')
    username = users.get(request.sid, 'Unknown')
    message_data = {'username': username, 'text': message, 'type': 'text'}

    if to:
        if to in users.values():
            recipient_sid = next(key for key, value in users.items() if value == to)
            private_messages[username].append(message_data)
            private_messages[to].append(message_data)
            emit('receive_message', message_data, room=recipient_sid)
        else:
            emit('receive_message', {'username': 'System', 'text': f'User {to} not found.', 'type': 'text'}, room=request.sid)
    else:
        public_chat_history.append(message_data)
        emit('receive_message', message_data, broadcast=True)

@socketio.on('send_image')
def handle_image(data):
    url = data['url']
    to = data.get('to')
    username = users.get(request.sid, 'Unknown')
    image_data = {'username': username, 'url': url, 'type': 'image'}

    if to:
        if to in users.values():
            recipient_sid = next(key for key, value in users.items() if value == to)
            private_messages[username].append(image_data)
            private_messages[to].append(image_data)
            emit('receive_image', image_data, room=recipient_sid)
        else:
            emit('receive_message', {'username': 'System', 'text': f'User {to} not found.', 'type': 'text'}, room=request.sid)
    else:
        public_chat_history.append(image_data)
        emit('receive_image', image_data, broadcast=True)

@socketio.on('disconnect')
def handle_disconnect():
    username = users.pop(request.sid, None)
    if username:
        user_order.remove(username)
        emit('user_list', user_order, broadcast=True)

@app.route('/upload_image', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return {'error': 'No file part'}
    file = request.files['image']
    if file.filename == '':
        return {'error': 'No selected file'}
    if file:
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        url = f'/uploads/{filename}'
        return {'url': url}
    return {'error': 'File upload failed'}

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

if __name__ == '__main__':
    clear_uploaded_images()
    port = 5000
    public_url = ngrok.connect(port)
    print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, port))
    socketio.run(app, port=port)    

 * ngrok tunnel "NgrokTunnel: "https://bfed-20-192-21-48.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000/"


In [3]:
import os
from flask import Flask, request, send_from_directory, render_template_string
from flask_socketio import SocketIO, emit
from werkzeug.utils import secure_filename
from pyngrok import ngrok

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

if not os.path.exists(app.config['UPLOAD_FOLDER']):
    os.makedirs(app.config['UPLOAD_FOLDER'])

public_chat_history = []
private_messages = {}
users = {}
user_order = []

def clear_uploaded_images():
    for filename in os.listdir(app.config['UPLOAD_FOLDER']):
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        os.remove(file_path)

chat_html = '''<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <link href="https://unpkg.com/@bentoproject/core@0.2.3/dist/styles.css" rel="stylesheet">
  <title>Spider-Chat</title>
  <script src="https://cdn.socket.io/4.0.1/socket.io.min.js"></script>
  <style>
    body {
      font-family: 'Chakra Petch', sans-serif;
      margin: 0;
      display: flex;
      flex-direction: column;
      height: 100vh;
      background-color: #fef7dadc;
      color: #ffffff;
    }
    .chat-container {
      flex: 1;
      display: flex;
      flex-direction: column;
      padding: 10px;
    }
    #header {
      color: white;
      text-align: center;
      padding: 10px 0;
      font-size: 24px;
      position:sticky;
      top: 46px;
      z-index: 10000;
    }
    #hamburger {
      position: absolute;
      top: 50%;
      left: 10px;
      transform: translateY(-50%);
      cursor: pointer;
      z-index: 10000;
      width: 50px;
      height: 50px;
      display: flex;
      align-items: center;
      justify-content: center;
      border-radius: 50%;
      background: black;
      transition: all 0.3s ease;
    }
    .line {
      position: absolute;
      height: 2px;
      background-color: white;
      transition: all 0.3s ease;
    }
    .line1 {
      width: 20px;
      top: 40%;
    }
    .line2 {
      width: 10px;
      top: 60%;
    }
    #hamburger.open .line1 {
      transform: rotate(45deg);
      width: 20px;
      top: 50%;
    }
    #hamburger.open .line2 {
      transform: rotate(-45deg);
      width: 20px;
      top: 50%;
    }
    #messages {
      flex: 1;
      overflow-y: auto;
      margin: 10px 0;
    }
    .input-container {
      display: flex;
      padding: 10px 0;
      position:sticky;
      bottom:15px;
      background-color:#fef7da;
    }
    #myMessage {
      flex: 1;
      padding: 10px;
      border: 1px solid #333;
      border-radius: 40px;
      margin-right: 10px;
      background: #1e1e1e;
      color: #fff;
    }
    #sendbutton {
      background: linear-gradient(135deg, #d7d1b7, #000);
      color: white;
      border: none;
      padding: 10px;
      border-radius: 4px;
      cursor: pointer;
    }
    #attachment {
      background: linear-gradient(135deg, #d7d1b7, #000);
      color: white;
      border: none;
      padding: 10px;
      border-radius: 4px;
      cursor: pointer;
      margin-right: 10px;
    }
    .navbar {
      background-color: #000000;
      border-left: 1px solid #333;
      position: fixed;
      bottom: -100%;
      width: 100%;
      height: 82%;
      transition: bottom ease-in-out 0.3s;
      z-index: 1000;
      border-radius: 50px;
      border-bottom-left-radius: 0;
      border-bottom-right-radius: 0;
      color: #fef7da;
    }
    .navbar.open {
      bottom: 0;
    }
    h2{
      color: #fef7da;
    }
    .navbar h2 {
      text-align: center;
      padding: 10px 0;
      margin: 10px;
      z-index: 1000;
      text-align: center;
      padding: 20px;
      margin-top: 20px;
      border:3px solid #fef7da;
      border-radius: 50px;
      color: #fef7da;
      font-size: 22px;
      font-weight: 700;
    }
    #userList {
      list-style: none;
      padding: 0;
      margin: 0;
    }
    #userList li {
      padding: 10px;
      cursor: pointer;
      border-bottom: 1px solid #333;
      color: #fff;
    }
    .message {
      display: flex;
      align-items: center;
      margin-bottom: 10px;
    }
    .message .profile-pic {
      width: 40px;
      height: 40px;
      margin-right: 10px;
    }
    .message .profile-pic img {
      width: 100%;
      border-radius: 50%;
    }
    @media (max-width: 768px) {
      .navbar {
        width: 100%;
      }
      #header {
        font-size: 28px;
      }
      #myMessage {
        font-size: 18px;
      }
      #sendbutton {
        font-size: 18px;
      }
      #userList li {
        font-size: 18px;
      }
      .message {
        font-size: 18px;
      }
    }
    .texter {
      text-align: center;
      padding: 20px;
      margin-top: -10px;
      border:3px solid #000000;
      border-radius: 50px;
      color: #000;
      font-size: 22px;
      font-weight: 700;
      position:sticky;
      top:20px;
      background-color: #fef7da;
    }
    div {
      color:#000;
    }
  </style>
</head>
<body>
  <div class="chat-container">
    <div id="header">
      <div id="hamburger">
        <div class="line line1"></div>
        <div class="line line2"></div>
      </div>
    </div>
    <div class="texter">
      Spider-Chat
    </div>
    <div id="messages"></div>
    <div class="input-container">
      <button id="attachment"><i class="fas fa-paperclip"></i></button>
      <input id="myMessage" placeholder="Type here" autocomplete="off">
      <button id="sendbutton"><i class="fas fa-paper-plane"></i></button>
      <input type="file" id="fileInput" style="display: none;" accept="image/*">
    </div>
  </div>
  <div class="navbar" id="navbar">
    <h2>Users</h2>
    <ul id="userList"></ul>
  </div>
  <script type="text/javascript">
    document.addEventListener('DOMContentLoaded', (event) => {
      var socket = io.connect(window.location.href);
      var navbar = document.getElementById('navbar');
      var hamburger = document.getElementById('hamburger');
      var messageInput = document.getElementById('myMessage');

      function sendMessage(message, to) {
        if (message.trim() !== '') {
          const data = { message: message.trim(), to: to };
          socket.emit('send_message', data);
        }
      }

      function sendImage(file, to) {
        const formData = new FormData();
        formData.append('image', file);
        formData.append('to', to);
        fetch('/upload_image', {
          method: 'POST',
          body: formData
        }).then(response => response.json())
          .then(data => {
            if (data.url) {
              socket.emit('send_image', { url: data.url, to: to });
            }
          });
      }

      function renderMessage(message) {
        const messageElement = document.createElement('div');
        messageElement.classList.add('message');
        messageElement.innerHTML = `<div class="profile-pic"><img src="https://cdn.pixabay.com/photo/2015/10/05/22/37/blank-profile-picture-973460_960_720.png" /></div>
                                    <div><strong>${message.user}</strong>: ${message.text}</div>`;
        document.getElementById('messages').appendChild(messageElement);
        document.getElementById('messages').scrollTop = document.getElementById('messages').scrollHeight;
      }

      function renderPrivateMessage(message, fromUser) {
        const messageElement = document.createElement('div');
        messageElement.classList.add('message');
        messageElement.innerHTML = `<div class="profile-pic"><img src="https://cdn.pixabay.com/photo/2015/10/05/22/37/blank-profile-picture-973460_960_720.png" /></div>
                                    <div><strong>${fromUser}</strong>: ${message.text}</div>`;
        document.getElementById('privateMessages').appendChild(messageElement);
        document.getElementById('privateMessages').scrollTop = document.getElementById('privateMessages').scrollHeight;
      }

      function toggleNavbar() {
        navbar.classList.toggle('open');
        hamburger.classList.toggle('open');
      }

      hamburger.addEventListener('click', toggleNavbar);

      document.getElementById('sendbutton').addEventListener('click', () => {
        sendMessage(messageInput.value, null);
        messageInput.value = '';
      });

      document.getElementById('attachment').addEventListener('click', () => {
        document.getElementById('fileInput').click();
      });

      document.getElementById('fileInput').addEventListener('change', (event) => {
        if (event.target.files.length > 0) {
          sendImage(event.target.files[0], null);
        }
      });

      socket.on('receive_message', (data) => {
        renderMessage(data);
      });

      socket.on('update_users', (userList) => {
        const userListElement = document.getElementById('userList');
        userListElement.innerHTML = '';
        userList.forEach((user) => {
          const userElement = document.createElement('li');
          userElement.textContent = user;
          userElement.addEventListener('click', () => {
            openPrivateChat(user);
          });
          userListElement.appendChild(userElement);
        });
      });

      function openPrivateChat(username) {
        const privateChatWindow = window.open('', '_blank', 'width=400,height=600');
        privateChatWindow.document.write(`
          <!DOCTYPE html>
          <html>
          <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <link href="https://unpkg.com/@bentoproject/core@0.2.3/dist/styles.css" rel="stylesheet">
            <style>
              body {
                background-color: #333;
                color: #fff;
                font-family: Arial, sans-serif;
              }
              #privateMessages {
                height: 80vh;
                overflow-y: auto;
                padding: 10px;
              }
              #privateInputContainer {
                display: flex;
                padding: 10px;
                background-color: #222;
              }
              #privateMessageInput {
                flex: 1;
                padding: 10px;
                border: none;
                border-radius: 4px;
                background: #fff;
                color: #000;
              }
              #privateSendButton {
                background: #555;
                color: white;
                border: none;
                padding: 10px;
                border-radius: 4px;
                cursor: pointer;
                margin-left: 10px;
              }
            </style>
          </head>
          <body>
            <h2>Private Chat with ${username}</h2>
            <div id="privateMessages"></div>
            <div id="privateInputContainer">
              <input id="privateMessageInput" placeholder="Type your message" autocomplete="off">
              <button id="privateSendButton">Send</button>
            </div>
            <script src="https://cdn.socket.io/4.0.1/socket.io.min.js"></script>
            <script>
              const socket = io.connect(window.location.href);
              const privateMessages = document.getElementById('privateMessages');
              const privateMessageInput = document.getElementById('privateMessageInput');
              const privateSendButton = document.getElementById('privateSendButton');

              function sendPrivateMessage(message, toUser) {
                if (message.trim() !== '') {
                  const data = { message: message.trim(), to: toUser };
                  socket.emit('send_private_message', data);
                  privateMessageInput.value = '';
                }
              }

              privateSendButton.addEventListener('click', () => {
                sendPrivateMessage(privateMessageInput.value, '${username}');
              });

              socket.on('receive_private_message', (data) => {
                const messageElement = document.createElement('div');
                messageElement.innerHTML = `<strong>${data.from}</strong>: ${data.message}`;
                privateMessages.appendChild(messageElement);
                privateMessages.scrollTop = privateMessages.scrollHeight;
              });
        `);
      }

      socket.on('receive_private_message', (data) => {
        renderPrivateMessage(data, data.from);
      });

      socket.on('connect', () => {
        const username = prompt("Enter your username:");
        if (username) {
          socket.emit('new_user', username);
        }
      });
    });
  </script>
</body>
</html>'''

@app.route('/')
def index():
    return render_template_string(chat_html)

@socketio.on('send_message')
def handle_message(data):
    public_chat_history.append(data)
    emit('receive_message', data, broadcast=True)

@socketio.on('send_private_message')
def handle_private_message(data):
    recipient = data['to']
    if recipient in users:
        recipient_sid = users[recipient]
        emit('receive_private_message', {'from': request.sid, 'message': data['message']}, room=recipient_sid)

@socketio.on('new_user')
def handle_new_user(username):
    users[username] = request.sid
    user_order.append(username)
    emit('update_users', user_order, broadcast=True)

@app.route('/upload_image', methods=['POST'])
def upload_image():
    file = request.files['image']
    if file:
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)
        url = f'/uploads/{filename}'
        return {'url': url}

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print("Public URL:", public_url)
    socketio.run(app, port=5000)



Public URL: NgrokTunnel: "https://f51d-20-192-21-48.ngrok-free.app" -> "http://localhost:5000"
